In [52]:
from collections import namedtuple

import matplotlib.pyplot as plt
import numpy as np
import PIL
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dset
from torch.utils.data.sampler import SubsetRandomSampler, Sampler

from torchvision import transforms

In [53]:
data_train = dset.SVHN('./data/', transform=transforms.Compose([transforms.ToTensor(),
                           transforms.Normalize(mean=[0.43,0.44,0.47],
                                               std=[0.20,0.20,0.20])]))
data_test = dset.SVHN('./data/', split='test', transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize(mean=[0.43,0.44,0.47],
                                               std=[0.20,0.20,0.20])
                       ]))

In [54]:
batch_size = 64
want_size = 1000

data_size = data_train.data.shape[0]
validation_split = .2
split = int(np.floor(validation_split * want_size))
indices = list(range(data_size))
np.random.shuffle(indices)
indices = indices[:want_size]

train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

val_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size,
                                         sampler=val_sampler)

In [55]:
def train_model(model, train_loader, val_loader, loss, optimizer, scheduler, num_epochs):
    loss_history = []
    train_history = []
    val_history = []
    for epoch in range(num_epochs):
        scheduler.step()
        model.train() # Enter train mode

        loss_accum = 0
        correct_samples = 0
        total_samples = 0
        for i_step, (x, y) in enumerate(train_loader):
            prediction = model(x)
            loss_value = loss(prediction, y)
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()

            _, indices = torch.max(prediction, 1)
            correct_samples += torch.sum(indices == y)
            total_samples += y.shape[0]

            loss_accum += loss_value

        ave_loss = loss_accum / i_step
        train_accuracy = float(correct_samples) / total_samples
        val_accuracy = compute_accuracy(model, val_loader)

        loss_history.append(float(ave_loss))
        train_history.append(train_accuracy)
        val_history.append(val_accuracy)
        print("epoch %d Average loss: %f, Train accuracy: %f, val accuracy: %f" %
                  (epoch+1, ave_loss, train_accuracy, val_accuracy))
        #if train_accuracy < 0.15:
        #    return loss_history, train_history, val_history

    return loss_history, train_history, val_history

def compute_accuracy(model, loader):
    """
    Computes accuracy on the dataset wrapped in a loader

    Returns: accuracy as a float value between 0 and 1
    """
    model.eval()
    correct_samples = 0
    total_samples = 0
    for i_step, (x, y) in enumerate(loader):
        # x_gpu = x.to(device)
        # y_gpu = y.to(device)
        prediction = model(x)
        _, indices = torch.max(prediction, 1)
        correct_samples += torch.sum(indices == y)
        total_samples += y.shape[0]

    train_accuracy = float(correct_samples) / total_samples
    return train_accuracy

#loss_history, train_history, val_history = train_model(nn_model, train_loader, val_loader, loss, optimizer, 5)

In [56]:
# We'll use a special helper module to shape it into a flat tensor
class Flattener(nn.Module):
    def forward(self, x):
        batch_size, *_ = x.shape
        return x.view(batch_size, -1)

In [95]:
Hyperparams = namedtuple("Hyperparams", ['transforms'])
RunResult = namedtuple("RunResult", ['val_history', 'final_val_accuracy'])
run_record = {}

In [98]:

epoch_num = 20
loss = nn.CrossEntropyLoss().type(torch.FloatTensor)

tfs = transforms.Compose([
    transforms.ColorJitter(hue=.20, saturation=.20),
    transforms.RandomRotation(10),
    transforms.RandomAdjustSharpness(3, p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.43,0.44,0.47],
                       std=[0.20,0.20,0.20])
])
data_train = dset.SVHN('./data/', transform = tfs)
train_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size,
                                           sampler=train_sampler)

for i in range(10):
    my_model = nn.Sequential(
      nn.Conv2d(3, 6, 5, padding=0),
      nn.ReLU(inplace=True),
      nn.BatchNorm2d(6),
      nn.MaxPool2d(2),
      nn.Conv2d(6, 16, 5, padding=0),
      nn.ReLU(inplace=True),
      nn.BatchNorm2d(16),
      nn.MaxPool2d(2),
      Flattener(),
      nn.Linear(16*5*5, 120),
      nn.ReLU(inplace=True),
      nn.BatchNorm1d(120),
      nn.Linear(120, 84),
      nn.ReLU(inplace=True),
      nn.BatchNorm1d(84),
      nn.Linear(84, 10)
      )

    #lr = 10**np.random.uniform(-5, -1)
    lr = 0.0061
    reg = 3.4e-5
    gamma = 0.5
    step_size = 4
    print(tfs)


    optimizer = optim.Adam(my_model.parameters(), lr=lr, weight_decay=reg)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
    loss_history, train_history, val_history = train_model(my_model, train_loader, val_loader, loss, optimizer, scheduler, epoch_num)
    hp = Hyperparams(transforms=tfs)
    rr = RunResult(val_history=val_history, final_val_accuracy=val_history[-1])
    run_record[hp] = rr

Compose(
    ColorJitter(brightness=None, contrast=None, saturation=[0.8, 1.2], hue=[-0.2, 0.2])
    RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
    RandomAdjustSharpness(sharpness_factor=3,p=0.5)
    ToTensor()
    Normalize(mean=[0.43, 0.44, 0.47], std=[0.2, 0.2, 0.2])
)
epoch 1 Average loss: 2.521544, Train accuracy: 0.167500, val accuracy: 0.250000
epoch 2 Average loss: 2.080314, Train accuracy: 0.358750, val accuracy: 0.325000
epoch 3 Average loss: 1.636829, Train accuracy: 0.498750, val accuracy: 0.520000
epoch 4 Average loss: 1.206157, Train accuracy: 0.640000, val accuracy: 0.530000
epoch 5 Average loss: 0.986299, Train accuracy: 0.726250, val accuracy: 0.600000
epoch 6 Average loss: 0.762237, Train accuracy: 0.796250, val accuracy: 0.645000
epoch 7 Average loss: 0.573624, Train accuracy: 0.838750, val accuracy: 0.605000
epoch 8 Average loss: 0.444745, Train accuracy: 0.896250, val accuracy: 0.675000
epoch 9 Average loss: 0.351998, Train a

KeyboardInterrupt: 

In [ ]:
result = sorted(list(run_record.items()), key=lambda x: x[1].final_val_accuracy)[::-1]
for a, b in result:
    print(a, b.final_val_accuracy)

In [74]:
test_loader = torch.utils.data.DataLoader(data_test, batch_size=batch_size)
test_accuracy = compute_accuracy(my_model, test_loader)
print("Test accuracy: %2.4f" % test_accuracy)

Test accuracy: 0.6808
